In [1]:
import json
with open('../data/test_aware.json', 'r', encoding='utf-8') as f:
    j = json.loads(f.read())

pairs = []
for i in j:
    pairs.append([i["src"]+" "+i["tgt"], i["hyp"]])

print(pairs)

[['Sadly , at the end of his life , Luther became somewhat of a nutso . Crazy person ; crackpot or lunatic .', 'A crazy person .'], ["इतिहासकार लोगन FBI के पिछला नीति के बहुत आलोचना कइलन कि एजेंसी आसान केस पर ज़्यादा ध्यान देवेला अउरी आपन संसाधन खर्च करेला, खास कर कार चोरी जइसन केस पर, ताकि उनकर केस सुलझावे के दर ज़्यादा देखावल जा सके. Historians have criticized past FBI policies for focusing resources on cases which are easy to solve, especially stolen car cases, with the intent of boosting the agency's success rate.", "Historians have strongly criticized the FBI's previous policy that the agency focuses more on easy cases and spends its resources, especially on cases like car theft, to show a higher rate of resolution."], ['Àwọn erekùsù ìlà oòrùn adúláwọ̀ wà ní òkun India ní ìlà oòrùn adúláwọ̀. The East African Islands are in the Indian Ocean off the eastern coast of Africa.', 'The eastern islands of the Indian Ocean are located in the eastern part of the Indian Ocean.'], ["Smith was

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = AutoModelForSequenceClassification.from_pretrained('vectara/hallucination_evaluation_model').to(device)
# tokenizer = AutoTokenizer.from_pretrained('vectara/hallucination_evaluation_model')
model = AutoModelForSequenceClassification.from_pretrained('./model_dump').to(device)
tokenizer = AutoTokenizer.from_pretrained('./model_dump')

batch_size = 10
batches = [pairs[i:i + batch_size] for i in range(0, len(pairs), batch_size)]
scores = []
for pairs in batches:
    inputs = tokenizer.batch_encode_plus(pairs, return_tensors='pt', padding=True).to(device)
    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits.cpu().detach().numpy()
        # convert logits to probabilities
        score = 1 / (1 + np.exp(-logits)).flatten()
    scores.extend(score)


c:\Users\tangl\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\tangl\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
# from sentence_transformers import CrossEncoder
# model = CrossEncoder('vectara/hallucination_evaluation_model')

# scores = model.predict(pairs)

In [4]:
output = []
for s in scores:
    label = ""
    if s < 0.5:
        label = "Hallucination"
    else:
        label = "Not Hallucination"
    output.append({"label": label, "p(Hallucination)": str(s)})

print(output)
with open("output.json", 'w', encoding='utf-8') as f:
    json.dump(output, f)

[{'label': 'Not Hallucination', 'p(Hallucination)': '0.9975973'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9996575'}, {'label': 'Hallucination', 'p(Hallucination)': '0.0622725'}, {'label': 'Hallucination', 'p(Hallucination)': '0.14798091'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9997092'}, {'label': 'Hallucination', 'p(Hallucination)': '0.15094157'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9995796'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9166906'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9997106'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9996575'}, {'label': 'Hallucination', 'p(Hallucination)': '0.0055078347'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.99927694'}, {'label': 'Hallucination', 'p(Hallucination)': '0.06718899'}, {'label': 'Hallucination', 'p(Hallucination)': '0.01113014'}, {'label': 'Not Hallucination', 'p(Hallucination)': '0.9945608'}, {'label': 'Not Hallucination', 'p(Halluc